In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.optimizers import Adam
from keras.regularizers import l2
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras import regularizers
from tensorflow.python.keras import backend as K
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import MultiHeadAttention
from keras.layers import Input
from keras.layers import Lambda
from keras.layers import Concatenate,GlobalAveragePooling1D
from keras.layers import Masking, LSTM, TimeDistributed
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
import smote_variants as sv

In [2]:
data1 = pd.read_csv("dataset/camel14.csv")
(row, cell) = data1.values.shape
data2=data1.values
X=np.array(data2).astype(float)  
#X =  sv.CURE_SMOTE()

In [3]:
y=X[:,cell-1]
X=X[:,0:cell-1]
X_res, y_res = X, y
X_res.shape,y_res.shape
X_res.shape
X_res=X_res.reshape(row,cell-1,1)
X_train, y_train = X_res, y_res

In [6]:
data1 = pd.read_csv("dataset/camel16.csv")
(row1, cell1) = data1.values.shape
data1.values.shape
data2=data1.values
X=np.array(data2).astype(float)  
#X =  sv.CURE_SMOTE()


In [7]:
y=X[:,cell-1]
X=X[:,0:cell-1]
X_res, y_res = X, y
X_res.shape,y_res.shape
X_res.shape
X_res=X_res.reshape(row1,cell-1,1)
X_test, y_test = X_res, y_res

In [8]:
# 转换标签为独热编码
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1429, 20, 1), (934, 20, 1), (1429, 2), (934, 2))

In [12]:
# Define the search space
lb = [8, 2, 1, 1, 0.0001]  # Lower bounds for the hyperparameters
ub = [64, 5, 4, 4, 0.01]   # Upper bounds for the hyperparameters

In [13]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
threshold = 0.5  

def mcc_metric(y_true, y_pred):
    predicted = tf.cast(tf.greater(y_pred, threshold), tf.float32)
    true_pos = tf.math.count_nonzero(predicted * y_true)
    true_neg = tf.math.count_nonzero((predicted - 1) * (y_true - 1))
    false_pos = tf.math.count_nonzero(predicted * (y_true - 1))
    false_neg = tf.math.count_nonzero((predicted - 1) * y_true)
    x = tf.cast((true_pos + false_pos) * (true_pos + false_neg) 
        * (true_neg + false_pos) * (true_neg + false_neg), tf.float32)
    return tf.cast((true_pos * true_neg) - (false_pos * false_neg), tf.float32) / tf.sqrt(x)

In [14]:
num_classes=2

def baseline_model(filters, kernel_size, num_heads, key_dim):
    inputs = Input(shape=(cell-1, 1))
    print(inputs.shape)
    conv = Conv1D(filters=filters, kernel_size=int(kernel_size), activation='relu', kernel_regularizer='l2')(inputs)
    pool = MaxPooling1D(pool_size=2)(conv)
    multi_attention = MultiHeadAttention(num_heads=int(num_heads), key_dim=int(key_dim))(pool, pool)
    masked = Masking(mask_value=0.0)(multi_attention)
    lstm = LSTM(64, return_sequences=True)(masked)
    self_supervised_task = TimeDistributed(Dense(1))(lstm)
    flattened_self_supervised_task = Flatten()(self_supervised_task)
    flattened_multi_attention = Flatten()(multi_attention)
    concatenated = Concatenate()([flattened_multi_attention, flattened_self_supervised_task])
    fc = Dense(21, activation='relu', kernel_regularizer='l2')(concatenated)
    outputs = Dense(num_classes, activation='sigmoid', kernel_regularizer='l2')(fc)
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
def fitness_function(hyperparameters):
    filters, kernel_size, num_heads, key_dim, learning_rate = hyperparameters
    filters = int(filters)  # Ensure filters is an integer
    kernel_size = int(kernel_size)  # Ensure kernel_size is an integer
    num_heads = int(num_heads)  # Ensure num_heads is an integer
    key_dim = int(key_dim)  # Ensure key_dim is an integer
    model = baseline_model(filters, kernel_size, num_heads, key_dim)
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy', f1_m, recall_m, mcc_metric])
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=64, batch_size=64,verbose=0)
    mcc = history.history['val_mcc_metric'][-1]  
    return -mcc 

In [16]:
def pso(objective_function, lb, ub, swarmsize=10, maxiter=20, wmax=0.9,wmin=0.4, cmin=0, cmax=4.0):
    """
    Particle Swarm Optimization (PSO) algorithm.

    Parameters:
        objective_function: The objective function to be minimized. This function should take a single argument representing the particle's position and return the corresponding fitness value.
        lb: Lower bounds for each dimension of the search space.
        ub: Upper bounds for each dimension of the search space.
        swarmsize: Number of particles in the swarm (default=10).
        maxiter: Maximum number of iterations (default=100).
        w: Inertia weight (default=0.5).
        c1: Cognitive parameter (default=2.0).
        c2: Social parameter (default=2.0).

    Returns:
        global_best_position: The best solution found by the algorithm.
        global_best_fitness: The fitness value of the best solution found.
    """

    num_dimensions = len(lb)
    position = np.random.uniform(lb, ub, size=(swarmsize, num_dimensions))
    velocity = np.zeros_like(position)
    personal_best_position = position.copy()
    personal_best_fitness = np.array([np.inf] * swarmsize)
    global_best_fitness = np.inf
    global_best_position = np.zeros(num_dimensions)

    for iter in range(maxiter):
        w = wmax - (wmax - wmin) * iter / maxiter
        t = w + 0.1
        c1 = cmin + (cmax - cmin) / (1 + np.exp(maxiter / 2 - iter))
        c2 = cmin + (cmax - cmin) / (1 + np.exp(iter - maxiter / 2))
        for i in range(swarmsize):
            fitness = objective_function(position[i])
            if fitness < personal_best_fitness[i]:
                personal_best_fitness[i] = fitness
                personal_best_position[i] = position[i].copy()
            if fitness < global_best_fitness:
                global_best_fitness = fitness
                global_best_position = position[i].copy()

        print("Iteration:", iter+1)
        print("Global best position:", global_best_position)
        print("Global best fitness:", global_best_fitness)

        for i in range(swarmsize):
            velocity[i] = w * velocity[i] + c1 * np.random.rand() * (personal_best_position[i] - position[i]) \
                           + c2 * np.random.rand() * (global_best_position - position[i])
            position[i] += (t * velocity[i])
            position[i] = np.clip(position[i], lb, ub)

    return global_best_position, global_best_fitness


In [17]:
# Enable eager execution
tf.config.run_functions_eagerly(True)

In [18]:
print("Start PSO optimization...")
best_hyperparameters, _ = pso(fitness_function, lb, ub, swarmsize=10, maxiter=20)
print("PSO optimization completed.")
print("Best hyperparameters found:", best_hyperparameters)


Start PSO optimization...
(None, 20, 1)


G:\anaconda3\Lib\site-packages\keras\src\layers\layer.py:915: UserWarning: Layer 'flatten' (of type Flatten) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
G:\anaconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


(None, 20, 1)


G:\anaconda3\Lib\site-packages\keras\src\layers\layer.py:915: UserWarning: Layer 'flatten_2' (of type Flatten) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


(None, 20, 1)


...

(None, 20, 1)


G:\anaconda3\Lib\site-packages\keras\src\layers\layer.py:915: UserWarning: Layer 'flatten_396' (of type Flatten) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


(None, 20, 1)


G:\anaconda3\Lib\site-packages\keras\src\layers\layer.py:915: UserWarning: Layer 'flatten_398' (of type Flatten) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Iteration: 20
Global best position: [2.61073572e+01 2.74895218e+00 4.00000000e+00 3.59059912e+00
 3.29127638e-03]
Global best fitness: -0.5099780559539795
PSO optimization completed.
Best hyperparameters found: [2.61073572e+01 2.74895218e+00 4.00000000e+00 3.59059912e+00
 3.29127638e-03]


In [19]:
# Build the final model with best hyperparameters
filters, kernel_size, num_heads, key_dim, learning_rate = best_hyperparameters
filters = int(filters)  # Ensure filters is an integer
kernel_size = int(kernel_size)  # Ensure kernel_size is an integer
num_heads = int(num_heads)  # Ensure num_heads is an integer
key_dim = int(key_dim)  # Ensure key_dim is an integer
final_model = baseline_model(filters, kernel_size, num_heads, key_dim)
final_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy', f1_m, recall_m, mcc_metric])


(None, 20, 1)


G:\anaconda3\Lib\site-packages\keras\src\layers\layer.py:915: UserWarning: Layer 'flatten_400' (of type Flatten) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [30]:
# Train the final model
history = final_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=64, batch_size=64, verbose=1)


Epoch 1/64
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 266ms/step - accuracy: 0.7294 - f1_m: 0.7283 - loss: 0.5328 - mcc_metric: 0.4581 - recall_m: 0.7261 - val_accuracy: 0.5899 - val_f1_m: 0.5864 - val_loss: 0.7028 - val_mcc_metric: 0.1820 - val_recall_m: 0.5802
Epoch 2/64
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 246ms/step - accuracy: 0.7453 - f1_m: 0.7425 - loss: 0.5524 - mcc_metric: 0.4882 - recall_m: 0.7383 - val_accuracy: 0.7002 - val_f1_m: 0.6948 - val_loss: 0.6488 - val_mcc_metric: 0.3883 - val_recall_m: 0.6962
Epoch 3/64
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 249ms/step - accuracy: 0.7470 - f1_m: 0.7482 - loss: 0.5369 - mcc_metric: 0.4963 - recall_m: 0.7480 - val_accuracy: 0.5567 - val_f1_m: 0.5525 - val_loss: 0.6878 - val_mcc_metric: 0.1087 - val_recall_m: 0.5503
Epoch 4/64
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 245ms/step - accuracy: 0.6962 - f1_m: 0.6946 - loss: 0.5822 - mcc_metric: 0.3927 - recall_m: 0.6903 - val_accuracy: 0.5493 - val_f1_m: 0.5432 - val_loss: 0.6713 - val_mcc_metric: 0.0980 - val_recall_m: 0.5365
